In [ ]:


from __future__ import annotations
import os, json, csv
from dataclasses import dataclass, asdict
from typing import List, Dict, Optional, Tuple
from datetime import datetime, date

import tkinter as tk
from tkinter import ttk, messagebox

# ---------------------
# Paths & persistence
# ---------------------
BASE_DIR = os.path.dirname(__file__) if '__file__' in globals() else os.getcwd()
DATA_DIR = os.path.join(BASE_DIR, 'data')
os.makedirs(DATA_DIR, exist_ok=True)
CSV_PATH = os.path.join(DATA_DIR, 'wellness_log.csv')  # V1 personal log (separate from V2 Kaggle CSV)
STATE_PATH = os.path.join(DATA_DIR, 'model_state.json')

def get_v1_paths() -> Tuple[str, str]:

    return CSV_PATH, STATE_PATH


def load_engine() -> "BeliefEngine":
    eng = BeliefEngine(); eng.load_state(); return eng

# ---------------------
# Utilities
# ---------------------
def clamp(v, lo, hi):
    return max(lo, min(hi, v))

def ewma(prev: float, x: float, alpha: float = 0.2) -> float:
    return alpha * x + (1 - alpha) * prev

def s_float(v, default=0.0):
    try:
        return float(v)
    except Exception:
        return default

def s_int(v, default=0):
    try:
        return int(v)
    except Exception:
        return default

# ---------------------
# Belief State (Model)
# ---------------------
@dataclass
class RollingBaselines:
    sleep_h: float = 7.0
    resting_hr: float = 70.0
    steps: float = 6000.0
    water_ml: float = 1800.0

@dataclass
class ModelState:
    baselines: RollingBaselines
    last_scores: List[float]

    @staticmethod
    def default():
        return ModelState(RollingBaselines(), [])

    def to_json(self):
        return {"baselines": asdict(self.baselines), "last_scores": self.last_scores[-30:]}

    @staticmethod
    def from_json(obj):
        bl = obj.get('baselines', {})
        return ModelState(
            RollingBaselines(
                sleep_h=bl.get('sleep_h', 7.0),
                resting_hr=bl.get('resting_hr', 70.0),
                steps=bl.get('steps', 6000.0),
                water_ml=bl.get('water_ml', 1800.0),
            ),
            list(obj.get('last_scores', []))[:30]
        )

class BeliefEngine:
    """Maintains rolling baselines (EWMA), computes risk indices & Wellness Score, plans actions."""
    def __init__(self, state: Optional[ModelState] = None):
        self.state = state or ModelState.default()
        self.log: List[str] = []

    # ---- internals ----
    def _log(self, msg: str):
        ts = datetime.now().strftime('%H:%M:%S')
        self.log.append(f"[{ts}] {msg}")
        if len(self.log) > 200:
            self.log.pop(0)

    # ---- compute metrics ----
    def compute(self, sleep_h: float, resting_hr: float, steps: float, water_ml: float,
                stress_1to5: int, caffeine: bool) -> Dict[str, float]:
        bl = self.state.baselines
        # Normalize vs baselines (bounded)
        sleep_norm = clamp(sleep_h / max(1e-6, bl.sleep_h), 0.0, 1.5)
        hr_norm    = clamp(bl.resting_hr / max(1.0, resting_hr), 0.5, 1.5)  # lower hr is better
        steps_norm = clamp(steps / max(1.0, bl.steps), 0.3, 1.6)
        water_norm = clamp(water_ml / max(1.0, bl.water_ml), 0.3, 1.3)
        stress_norm= clamp((6 - stress_1to5) / 5.0, 0.0, 1.0)  # 1 best, 5 worst

        # Risks
        hydration_risk = clamp(1.0 - water_norm, 0.0, 1.5)
        fatigue_risk   = clamp((1.0 - sleep_norm) + 0.5*(1.0 - steps_norm) + 0.5*(1.0 - hr_norm), 0.0, 1.5)
        stress_risk    = clamp((1.0 - stress_norm) + 0.2*(1.0 - steps_norm), 0.0, 1.5)
        if caffeine:
            hydration_risk = clamp(hydration_risk + 0.1, 0.0, 1.5)
            stress_risk    = clamp(stress_risk + 0.1, 0.0, 1.5)

        # Score (0–100)
        penalty = (
            (1.0 - sleep_norm)*22 + (1.0 - hr_norm)*18 + (1.0 - steps_norm)*24 +
            (1.0 - water_norm)*16 + (1.0 - stress_norm)*20
        )
        score = clamp(100.0 - max(0.0, penalty), 0.0, 100.0)

        self._log(f"Inputs sleep={sleep_h:.1f}h, HR={resting_hr:.0f}, steps={steps:.0f}, water={water_ml:.0f}ml, stress={stress_1to5}, caffeine={caffeine}")
        self._log(f"Norms sleep={sleep_norm:.2f}, hr={hr_norm:.2f}, steps={steps_norm:.2f}, water={water_norm:.2f}, stress={stress_norm:.2f}")
        self._log(f"Risks hydration={hydration_risk:.2f}, fatigue={fatigue_risk:.2f}, stress={stress_risk:.2f}")
        self._log(f"Score {score:.1f}")

        return {
            'score': score,
            'sleep_norm': sleep_norm,
            'hr_norm': hr_norm,
            'steps_norm': steps_norm,
            'water_norm': water_norm,
            'stress_norm': stress_norm,
            'hydration_risk': hydration_risk,
            'fatigue_risk': fatigue_risk,
            'stress_risk': stress_risk,
        }

    # ---- planner ----
    def plan_actions(self, metrics: Dict[str, float], inputs: Dict[str, float]):
        recs: List[Tuple[str, str, float]] = []
        if metrics['hydration_risk'] > 0.25:
            deficit = clamp(1.0 - metrics['water_norm'], 0.0, 1.0)
            add_ml = int(500 + 1500*deficit)
            recs.append((f"Drink +{add_ml} ml water", "Hydration below baseline.", 4.5))
        if metrics['fatigue_risk'] > 0.4:
            if inputs['sleep_h'] < 7:
                recs.append(("Aim for 7–8h sleep tonight", "Sleep below optimal; fatigue risk.", 6.0))
            if inputs['steps'] < max(4000, self.state.baselines.steps*0.8):
                recs.append(("Take a 20–25 min brisk walk", "Activity below baseline.", 5.0))
        if metrics['stress_risk'] > 0.35 or inputs['stress_1to5'] >= 4:
            recs.append(("Do a 5–7 min breathing session", "Stress elevated; quick relief.", 4.0))
            recs.append(("Reduce screens 30 min before bed", "Sleep hygiene supports recovery.", 3.0))
        if not recs:
            recs = [("Maintain routine", "Metrics near baseline.", 1.0)]
        recs.sort(key=lambda x: x[2], reverse=True)
        # dedupe
        seen, final = set(), []
        for r in recs:
            if r[0] not in seen:
                final.append(r); seen.add(r[0])
        return final[:3]

    # ---- learning update ----
    def update_baselines(self, sleep_h: float, resting_hr: float, steps: float, water_ml: float, score: float):
        bl = self.state.baselines
        bl.sleep_h    = ewma(bl.sleep_h, sleep_h)
        bl.resting_hr = ewma(bl.resting_hr, resting_hr)
        bl.steps      = ewma(bl.steps, steps)
        bl.water_ml   = ewma(bl.water_ml, water_ml)
        self.state.last_scores.append(score)
        self.state.last_scores = self.state.last_scores[-30:]
        self._log(f"Baselines→ sleep={bl.sleep_h:.2f}h HR={bl.resting_hr:.1f} steps={bl.steps:.0f} water={bl.water_ml:.0f}ml")

    # ---- persistence ----
    def save_state(self):
        with open(STATE_PATH, 'w', encoding='utf-8') as f:
            json.dump(self.state.to_json(), f, indent=2)

    def load_state(self):
        if os.path.exists(STATE_PATH):
            with open(STATE_PATH, 'r', encoding='utf-8') as f:
                obj = json.load(f)
            self.state = ModelState.from_json(obj)
            self._log('Model state loaded.')
        else:
            self._log('No model_state.json; using defaults.')

# ---------------------
# CSV helpers
# ---------------------
CSV_HEADER = [
    'date','sleep_h','resting_hr','steps','water_ml','stress_1to5','caffeine','score','recs'
]

def csv_append(row: Dict[str,str]):
    exists = os.path.exists(CSV_PATH)
    with open(CSV_PATH, 'a', newline='', encoding='utf-8') as f:
        w = csv.DictWriter(f, fieldnames=CSV_HEADER)
        if not exists:
            w.writeheader()
        w.writerow(row)

def append_csv(row: Dict[str,str]):
    """Public alias for csv_append used by external callers (e.g., Streamlit Tab 1)."""
    return csv_append(row)

# ---------------------
# Tkinter GUI
# ---------------------
class HeliosGUI:
    def __init__(self, root):
        self.root = root
        self.root.title('HELIOS-AI — Wellness Advisor (V1, Notebook Edition)')
        self.engine = BeliefEngine(); self.engine.load_state()

        self._build()
        self._refresh_log()
        self._refresh_recent()

    def _build(self):
        self.root.geometry('1000x600')
        main = ttk.Frame(self.root, padding=10); main.pack(fill='both', expand=True)
        left = ttk.Frame(main); center = ttk.Frame(main); right = ttk.Frame(main)
        left.grid(row=0,column=0,sticky='nsew',padx=(0,10))
        center.grid(row=0,column=1,sticky='nsew',padx=(0,10))
        right.grid(row=0,column=2,sticky='nsew')
        for i in range(3): main.columnconfigure(i, weight=1)
        main.rowconfigure(0, weight=1)

        # Inputs
        box = ttk.LabelFrame(left, text="Today's Inputs"); box.pack(fill='x')
        self.var_sleep = tk.StringVar(value='7.0')
        self.var_hr    = tk.StringVar(value='70')
        self.var_steps = tk.StringVar(value='6000')
        self.var_water = tk.StringVar(value='1800')
        self.var_stress= tk.StringVar(value='3')
        self.var_caf   = tk.BooleanVar(value=False)

        def row(parent,label,var,unit=''):
            r=ttk.Frame(parent); r.pack(fill='x', pady=2)
            ttk.Label(r,text=label,width=16).pack(side='left')
            ttk.Entry(r,textvariable=var,width=12).pack(side='left')
            if unit: ttk.Label(r,text=unit).pack(side='left')

        row(box,'Sleep Hours', self.var_sleep,'h')
        row(box,'Resting HR', self.var_hr,'bpm')
        row(box,'Steps',      self.var_steps,'count')
        row(box,'Water Intake',self.var_water,'ml')
        srow=ttk.Frame(box); srow.pack(fill='x',pady=2)
        ttk.Label(srow,text='Stress (1–5)',width=16).pack(side='left')
        ttk.Combobox(srow,textvariable=self.var_stress,values=['1','2','3','4','5'],state='readonly',width=10).pack(side='left')
        crow=ttk.Frame(box); crow.pack(fill='x',pady=2)
        ttk.Checkbutton(crow,text='Caffeine today',variable=self.var_caf).pack(side='left')

        acts=ttk.Frame(left); acts.pack(fill='x',pady=8)
        ttk.Button(acts,text='Compute Wellness',command=self.on_compute).pack(fill='x',pady=2)
        ttk.Button(acts,text='Save Day',command=self.on_save).pack(fill='x',pady=2)
        ttk.Button(acts,text='Reset Baselines',command=self.on_reset).pack(fill='x',pady=2)
        ttk.Button(acts,text='Open CSV Location',command=self.on_open_csv).pack(fill='x',pady=2)

        # Center: Score + Log
        top = ttk.Frame(center); top.pack(fill='x')
        ttk.Label(top,text='Wellness Score',font=('Helvetica',16,'bold')).pack(side='left')
        self.score_var = tk.StringVar(value='—')
        ttk.Label(top,textvariable=self.score_var,font=('Helvetica',20)).pack(side='left',padx=10)

        log_box = ttk.LabelFrame(center,text='Agent Reasoning Trace'); log_box.pack(fill='both',expand=True, pady=6)
        self.log_text = tk.Text(log_box,height=18,wrap='word',state='disabled')
        self.log_text.pack(fill='both',expand=True)

        recent = ttk.LabelFrame(center,text='Recent Scores (last 7)'); recent.pack(fill='x', pady=6)
        self.recent_var = tk.StringVar(value='—')
        ttk.Label(recent,textvariable=self.recent_var).pack(anchor='w', padx=6, pady=6)

        # Right: Recs + Plan
        recs_box = ttk.LabelFrame(right,text="Today's Recommendations"); recs_box.pack(fill='both',expand=True,pady=6)
        self.recs_tree = ttk.Treeview(recs_box,columns=('rec','why','impact'),show='headings',height=8)
        self.recs_tree.heading('rec',text='Recommendation')
        self.recs_tree.heading('why',text='Rationale')
        self.recs_tree.heading('impact',text='Expected ΔScore')
        self.recs_tree.column('rec',width=180,anchor='w')
        self.recs_tree.column('why',width=320,anchor='w')
        self.recs_tree.column('impact',width=120,anchor='center')
        self.recs_tree.pack(fill='both',expand=True)

        plan = ttk.LabelFrame(right,text='Daily Plan Checklist'); plan.pack(fill='x',pady=6)
        self.plan_vars = [tk.BooleanVar(value=False) for _ in range(3)]
        self.plan_labels = []
        for i in range(3):
            cb = ttk.Checkbutton(plan,text=f'Task {i+1}',variable=self.plan_vars[i])
            cb.pack(anchor='w'); self.plan_labels.append(cb)

        ttk.Label(right,text='Disclaimer: Educational tool — not medical advice.',foreground='#a33').pack(anchor='w',pady=4)

    def _refresh_log(self):
        self.log_text.configure(state='normal')
        self.log_text.delete('1.0','end')
        for line in self.engine.log[-200:]:
            self.log_text.insert('end', line+'\n')
        self.log_text.configure(state='disabled'); self.log_text.see('end')

    def _refresh_recent(self):
        tail = self.engine.state.last_scores[-7:]
        if tail:
            bars = '  '.join(f"{int(s):02d}" for s in tail)
            self.recent_var.set(bars)
        else:
            self.recent_var.set('No history yet.')

    # ---- actions ----
    def on_compute(self):
        in_sleep = s_float(self.var_sleep.get(), 7.0)
        in_hr    = s_int(self.var_hr.get(), 70)
        in_steps = s_int(self.var_steps.get(), 6000)
        in_water = s_int(self.var_water.get(), 1800)
        in_stress= clamp(s_int(self.var_stress.get(), 3), 1, 5)
        in_caf   = bool(self.var_caf.get())

        metrics = self.engine.compute(in_sleep, in_hr, in_steps, in_water, in_stress, in_caf)
        self.score_var.set(f"{metrics['score']:.1f}")

        inputs = {
            'sleep_h': in_sleep,
            'resting_hr': in_hr,
            'steps': in_steps,
            'water_ml': in_water,
            'stress_1to5': in_stress,
            'caffeine': in_caf,
        }
        recs = self.engine.plan_actions(metrics, inputs)

        for item in self.recs_tree.get_children():
            self.recs_tree.delete(item)
        for rec, why, impact in recs:
            self.recs_tree.insert('', 'end', values=(rec, why, f"+{impact:.1f}"))

        # Show top recs as checklist labels
        for i, cb in enumerate(self.plan_labels):
            text = recs[i][0] if i < len(recs) else f'Task {i+1}'
            cb.config(text=text)
        for i in range(3):
            self.plan_vars[i].set(False)

        self._refresh_log()

    def on_save(self):
        try:
            if self.score_var.get() == '—':
                messagebox.showinfo('Info','Compute the wellness score first.'); return
            in_sleep = s_float(self.var_sleep.get(), 7.0)
            in_hr    = s_int(self.var_hr.get(), 70)
            in_steps = s_int(self.var_steps.get(), 6000)
            in_water = s_int(self.var_water.get(), 1800)
            in_stress= clamp(s_int(self.var_stress.get(), 3), 1, 5)
            in_caf   = bool(self.var_caf.get())
            score    = float(self.score_var.get())

            rec_texts = []
            for item in self.recs_tree.get_children():
                vals = self.recs_tree.item(item, 'values')
                if vals: rec_texts.append(vals[0])

            csv_append({
                'date': date.today().isoformat(),
                'sleep_h': f'{in_sleep:.1f}',
                'resting_hr': str(in_hr),
                'steps': str(in_steps),
                'water_ml': str(in_water),
                'stress_1to5': str(in_stress),
                'caffeine': str(int(in_caf)),
                'score': f'{score:.1f}',
                'recs': ' | '.join(rec_texts)
            })

            self.engine.update_baselines(in_sleep, in_hr, in_steps, in_water, score)
            self.engine.save_state()
            self._refresh_recent(); self._refresh_log()
            messagebox.showinfo('Saved','Day saved. Baselines updated.')
        except Exception as e:
            messagebox.showerror('Error', f'Failed to save day: {e}')

    def on_reset(self):
        self.engine.state = ModelState.default(); self.engine.save_state()
        self._refresh_recent(); self._refresh_log()
        messagebox.showinfo('Reset','Baselines reset to defaults.')

    def on_open_csv(self):
        if os.path.exists(CSV_PATH):
            messagebox.showinfo('CSV Location', CSV_PATH)
        else:
            messagebox.showinfo('CSV', 'No CSV yet — save at least one day.')

# ---------------------
# Launcher (run this cell to open the GUI window)
# ---------------------
if __name__ == '__main__':
    print('Notebook launcher disabled. Use: "streamlit run streamlit_app.py" (Tab: V1). Or run helios_ai.py for the standalone GUI.')